In [39]:
import pandas as pd
import numpy as np
import requests
import json
import random, time

In [3]:
### get_matches returns a dataframe from the opendota API with given parameters
### don't run this too often, as there is a request limit of one per second and 50000 pr. month
def get_matches(matches=10000, start_time_less_than=1594771200, mmr_lower=2000, mmr_upper=2500):
    # we build the base url for the opendota api 
    base = 'https://api.opendota.com/api/explorer'
    # we create the SQL percent encoded query. The query is as follows
    #"""
    #SELECT *
    #FROM public_matches
    #LEFT JOIN public_player_matches
    #ON public_matches.match_id = public_player_matches.match_id
    #WHERE lobby_type=7 AND game_mode=22 AND avg_mmr!=0 AND start_time<1594771200
    #ORDER BY start_time DESC
    #LIMIT 10000
    #"""
    matches_request = f'?sql=SELECT%20*%0AFROM%20public_matches%0ALEFT%20JOIN%20public_player_matches%0AON%20public_matches.match_id%20%3D%20public_player_matches.match_id%0AWHERE%20lobby_type%3D7%20AND%20game_mode%3D22%20AND%20avg_mmr!%3D0%20AND%20start_time<{start_time_less_than}%20AND%20avg_mmr>{mmr_lower}%20AND%20avg_mmr<{mmr_upper}%0AORDER%20BY%20start_time%20DESC%0ALIMIT%20{str(matches*10)}'
    
    url = base + matches_request

    result = requests.get(url)
    print(result)
    j_matches = result.json()

    return pd.DataFrame(j_matches["rows"])

In [4]:
practice_data_2000_2500 = get_matches(matches=10000, start_time_less_than=1594771200, mmr_lower=2000, mmr_upper=2500)

<Response [200]>


,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,player_slot,hero_id
0,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,132,47
1,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,131,8
2,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,130,21
3,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,129,84
4,5513436906,4623079558,True,1594771180,2121,2337,2,7,22,35,4,151,128,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5511274406,4621117016,False,1594646538,2033,2208,1,7,22,32,5,225,4,27
99996,5511274406,4621117016,False,1594646538,2033,2208,1,7,22,32,5,225,3,32
99997,5511274406,4621117016,False,1594646538,2033,2208,1,7,22,32,5,225,2,67
99998,5511274406,4621117016,False,1594646538,2033,2208,1,7,22,32,5,225,1,54


In [60]:
def practice_data(matches, sleep):
    mmr_steps = [i for i in range(1000, 8001) if i%1000 == 0]
    mmr_brackets = [[mmr_steps[i], mmr_steps[i+1]] for i in range(len(mmr_steps)) if mmr_steps[i] < 8000]
    dataframes = [get_matches(matches, start_time_less_than=1594771200, mmr_lower=lower, mmr_upper=upper) for lower,upper in mmr_brackets if time.sleep(sleep) is None]
    
    return dataframes
    
    #for lower,upper in mmr_brackets:
    #    practice_data_f"{lower}"_f"{upper}" = get_matches(matches=100, start_time_less_than=1594771200, mmr_lower=lower, mmr_upper=upper)
        
        

In [61]:
practice_data(100, 2)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


[       match_id  match_seq_num  radiant_win  start_time  duration  avg_mmr  \
 0    5513435609     4623088038         True  1594771115      2814     1359   
 1    5513435609     4623088038         True  1594771115      2814     1359   
 2    5513435609     4623088038         True  1594771115      2814     1359   
 3    5513435609     4623088038         True  1594771115      2814     1359   
 4    5513435609     4623088038         True  1594771115      2814     1359   
 ..          ...            ...          ...         ...       ...      ...   
 995  5513366808     4623006244        False  1594767333      1985     1944   
 996  5513366808     4623006244        False  1594767333      1985     1944   
 997  5513366808     4623006244        False  1594767333      1985     1944   
 998  5513366808     4623006244        False  1594767333      1985     1944   
 999  5513366808     4623006244        False  1594767333      1985     1944   
 
      num_mmr  lobby_type  game_mode  avg_rank_tie

In [58]:
practice_data_1000_2000 = get_matches(matches=50000, start_time_less_than=1594771200, mmr_lower=1000, mmr_upper=2000)

<Response [400]>


KeyError: 'rows'

In [ ]:
practice_data_2000_3000 = get_matches(matches=50000, start_time_less_than=1594771200, mmr_lower=2000, mmr_upper=3000)

In [ ]:
practice_data_3000_4000 = get_matches(matches=50000, start_time_less_than=1594771200, mmr_lower=3000, mmr_upper=4000)

In [ ]:
practice_data_4000_5000 = get_matches(matches=50000, start_time_less_than=1594771200, mmr_lower=4000, mmr_upper=5000)

In [ ]:
practice_data_5000_6000 = get_matches(matches=50000, start_time_less_than=1594771200, mmr_lower=5000, mmr_upper=6000)

In [ ]:
practice_data_6000_7000 = get_matches(matches=50000, start_time_less_than=1594771200, mmr_lower=6000, mmr_upper=7000)

In [ ]:
practice_data_7000_8000 = get_matches(matches=50000, start_time_less_than=1594771200, mmr_lower=7000, mmr_upper=8000)

In [29]:
mmr_steps = [i for i in range(10001) if i%500 == 0]
mmr_brackets = [[mmr_steps[i], mmr_steps[i+1]] for i in range(len(mmr_steps)) if mmr_steps[i] < 10000]

for lower,upper in mmr_brackets:
    print(upper)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
